---
title: "Winners and Losers"
format:
  html:
    code-fold: true
    code-summary: "Show the code"
  docx: 
    fig-height: 11.6 in
---

In [13]:
import pandas as pd
from fagrants_module import fagrants

def weighted_group_delta(grouped_df):
    return_dict = {}

    for year in range(2025, 2029):
        total_weight = grouped_df[f"ERP_{year}"].mean()

        return_dict[f"Population_{year}"] = total_weight

        weighted_mean = (
            grouped_df[f"Grant Delta_{year}"] * grouped_df[f"ERP_{year}"]
        ).sum() / grouped_df[f"ERP_{year}"].sum()

        return_dict[f"Grant Delta_{year}"] = weighted_mean

    return pd.Series(return_dict)

def grants_simulation(data, pc_min, pc_min_base):
    inst = fagrants.fagrants_model(
    data_path=data,
    per_capita_minimum=pc_min,
    per_capita_minimum_base=pc_min_base,
    )

    sim_grants = inst.run()

    sim_grants["Sim_i"] = sim_grants["UID"].str.split("_").str[1].astype(int)

    for year in range(2025, 2029):
        sim_grants[f"Grant Delta_{year}"] = (
            sim_grants[f"Grant_{year}"] - sim_grants[f"Grant_base_{year}"]
        )

        sim_grants[f"Grant Delta %_{year}"] = (
            sim_grants[f"Grant Delta_{year}"] * 100 / sim_grants[f"Grant_base_{year}"]
        )

    return sim_grants

def grants_delta(sim_grants):
    sim_grants_grouped = (
        sim_grants.groupby(["Jurisdiction", "ACLG", "LGA"])
        .apply(weighted_group_delta, include_groups=False)
        .reset_index()
    )

    pop_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Population_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Population",
    )

    sim_grants_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Grant Delta_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG ($)",
    )

    for year in range(2025, 2029):
        sim_grants_grouped[f"Grant Delta Per Capita_{year}"] = sim_grants_grouped[f"Grant Delta_{year}"]/ sim_grants_grouped[f"Population_{year}"]

    sim_grants_melted_per_capita = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction", "ACLG", "LGA"],
        value_vars=[f"Grant Delta Per Capita_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG Per Capita ($)",
    )

    sim_grants_melted_per_cent = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction", "ACLG", "LGA"],
        value_vars=[f"Grant Delta %_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG (%)",
    )

    sim_grants_melted["Year"] = sim_grants_melted["Year"].str.split("_").str[1].astype(int)

    sim_grants_melted["Expected Change in GPG (%)"] = sim_grants_melted_per_cent["Expected Change in GPG (%)"]

    sim_grants_melted["Expected Change in GPG Per Capita ($)"] = sim_grants_melted_per_capita["Expected Change in GPG Per Capita ($)"]

    sim_grants_melted["Population"] = pop_melted["Population"]

    return sim_grants_melted

In [12]:
sim_grants_30 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.30, 0.30)

sim_grants_20 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.20, 0.30)

sim_grants_10 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.10, 0.30)

sim_grants_0 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.00, 0.30)

In [14]:
sim_grants_30_melted = grants_delta(sim_grants_30)

sim_grants_20_melted = grants_delta(sim_grants_20)

sim_grants_10_melted = grants_delta(sim_grants_10)

sim_grants_0_melted = grants_delta(sim_grants_0)

KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['Grant Delta %_2025', 'Grant Delta %_2026', 'Grant Delta %_2027', 'Grant Delta %_2028']"

In [4]:
from datetime import datetime as tm

with pd.ExcelWriter(f"./Data/Output/Simulation Results {tm.today().strftime("%d_%m_%Y")}.xlsx") as writer:
    sim_grants_30_melted.to_excel(writer, sheet_name = "30% Minimum Grant")
    sim_grants_20_melted.to_excel(writer, sheet_name = "20% Minimum Grant")
    sim_grants_10_melted.to_excel(writer, sheet_name = "10% Minimum Grant")
    sim_grants_0_melted.to_excel(writer, sheet_name = "0% Minimum Grant")

In [7]:
sim_grants_20_melted

,Jurisdiction,ACLG,LGA,Year,Expected Change in GPG ($),Expected Change in GPG Per Capita($),Population
0,New South Wales,RAL,Berrigan Shire Council,2025,590533.302283,66.299910,8907.0
1,New South Wales,RAL,Bland Shire Council,2025,896005.712362,159.120176,5631.0
2,New South Wales,RAL,Blayney Shire Council,2025,289411.914520,36.736724,7878.0
3,New South Wales,RAL,Dungog Shire Council,2025,237856.667589,23.629711,10066.0
4,New South Wales,RAL,Edward River Council,2025,706615.491527,81.332354,8688.0
...,...,...,...,...,...,...,...
2131,Western Australia,URM,City of Busselton,2028,-450029.403391,-9.308706,48345.0
2132,Western Australia,URM,City of Greater Geraldton,2028,498564.977108,10.818144,46086.0
2133,Western Australia,URM,City of Kalgoorlie-Boulder,2028,30736.366253,1.016297,30243.5
2134,Western Australia,URS,Shire of Capel,2028,122821.725775,5.777399,21259.0


In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 1)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

def display_data(data, state):
    sub = data[data["Jurisdiction"] == state]

    k = sub.groupby(["Jurisdiction", "ACLG", "Year"]).agg(
        {
            "Expected Change in GPG ($)": lambda x: sum(x * sub.loc[x.index, "Population"])/sub.loc[x.index, "Population"].sum(),
            "Expected Change in GPG Per Capita ($)": lambda x: sum(x * sub.loc[x.index, "Population"])/sub.loc[x.index, "Population"].sum(),
            "Population": 'sum'
        }
    )

    display(k)

## A 20% Per Capita Minimum Grant

### New South Wales

In [9]:
display_data(sim_grants_20_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                  $ 557432.2   
                     2026                  $ 578379.8   
                     2027                  $ 599436.6   
                     2028                  $ 620824.9   
                RAM  2025                  $ 524900.9   
                     2026                  $ 544263.6   
                     2027                  $ 563143.5   
                     2028                  $ 583560.4   
                RAS  2025                  $ 726147.9   
                     2026                  $ 751728.5   
                     2027                  $ 778709.8   
                     2028                  $ 807973.8   
                RAV  2025                 $ -375144.5   
                     2026                 $ -389486.8   
                     2027                 $ -403160.5   
                     2028                 $ -417737.2   
                RTL  2025                  $ 827068.2   
                     2026                  $ 852107.1   
                     2027                  $ 884956.8   
                     2028                  $ 921182.6   
                RTM  2025                  $ 918745.9   
                     2026                  $ 954280.4   
                     2027                  $ 987834.1   
                     2028                 $ 1017947.1   
                RTX  2025                   $ 38607.2   
                     2026                   $ 40285.2   
                     2027                   $ 41674.2   
                     2028                   $ 43139.3   
                UCC  2025                $ -2066291.8   
                     2026                $ -2143027.5   
                     2027                $ -2221145.8   
                     2028                $ -2301203.6   
                UDL  2025                 $ -706370.6   
                     2026                 $ -732603.9   
                     2027                 $ -759310.0   
                     2028                 $ -786676.4   
                UDM  2025                 $ -390685.2   
                     2026                 $ -405192.6   
                     2027                 $ -419957.3   
                     2028                 $ -435066.8   
                UDS  2025                 $ -113072.1   
                     2026                 $ -117263.2   
                     2027                 $ -121566.7   
                     2028                 $ -125894.5   
                UDV  2025                $ -2195948.2   
                     2026                $ -2277724.8   
                     2027                $ -2361026.0   
                     2028                $ -2445938.8   
                UFL  2025                 $ -633006.3   
                     2026                 $ -656771.0   
                     2027                 $ -680143.6   
                     2028                 $ -704927.6   
                UFM  2025                 $ -368851.6   
                     2026                 $ -382348.3   
                     2027                 $ -396354.5   
                     2028                 $ -409278.3   
                UFV  2025                $ -1184403.7   
                     2026                $ -1229720.5   
                     2027                $ -1270643.4   
                     2028                $ -1315440.0   
                URL  2025                  $ 140243.5   
                     2026                  $ 147616.2   
                     2027                  $ 152045.1   
                     2028                  $ 158383.2   
                URM  2025                  $ 265683.3   
                     2026                  $ 273869.3   
                     2027                  $ 283252.1   
                     2028                  $ 293631.6   
                URS  2025            

### Victoria

In [72]:
display_data(sim_grants_20_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                    420266.2   
                  2026                    437806.8   
                  2027                    455077.4   
                  2028                    473115.2   
             RAM  2025                    471519.5   
                  2026                    495782.4   
                  2027                    518420.8   
                  2028                    539026.3   
             RAV  2025                    399883.1   
                  2026                    416340.0   
                  2027                    432232.2   
                  2028                    449620.9   
             UCC  2025                  -1599211.1   
                  2026                  -1665583.7   
                  2027                  -1732737.4   
                  2028                  -1801452.5   
             UDL  2025                   -824613.5   
                  2026                   -858539.8   
                  2027                   -893137.9   
                  2028                   -928294.6   
             UDV  2025                   -978069.8   
                  2026                  -1017110.8   
                  2027                  -1058758.2   
                  2028                  -1099339.4   
             UFL  2025                    306672.2   
                  2026                    312803.6   
                  2027                    332169.8   
                  2028                    349943.9   
             UFM  2025                     -2383.7   
                  2026                     -3165.5   
                  2027                     -1469.9   
                  2028                     -2422.7   
             UFS  2025                    301675.0   
                  2026                    313804.0   
                  2027                    325764.6   
                  2028                    336995.7   
             UFV  2025                   -103505.9   
                  2026                   -108919.6   
                  2027                   -115099.2   
                  2028                   -120785.8   
             URL  2025                    791482.0   
                  2026                    820296.9   
                  2027                    854266.1   
                  2028                    881226.1   
             URM  2025                    669540.5   
                  2026                    697216.7   
                  2027                    726403.9   
                  2028                    753835.6   
             URS  2025                    564652.4   
                  2026                    590875.0   
                  2027                    614668.0   
                  2028                    639034.9   
             URV  2025                    198055.0   
                  2026                    200949.8   
                  2027                    206541.1   
                  2028                    224136.0   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Victoria     RAL  2025                                  57.2     52851.0  
                  2026                                  58.6     53684.0  
                  2027                                  60.1     54451.0  
                  2028                                  61.6     55232.0  
             RAM  2025                                 114.9      4102.0  
                  2026                                 119.0      4167.0  
                  2027                                 122.6      4227.0  
                  2028                                 125.7      4287.0  
             RAV  2025                                  24.7    258681.0  
                  2026                                  25.3    26

### Queensland

In [73]:
display_data(sim_grants_20_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   1346398.9   
                  2026                   1401996.4   
                  2027                   1462959.0   
                  2028                   1527815.6   
             RAV  2025                    830760.1   
                  2026                    865335.9   
                  2027                    900036.0   
                  2028                    933630.4   
             RTL  2025                   1161342.1   
                  2026                   1208664.5   
                  2027                   1252877.3   
                  2028                   1302594.8   
             RTM  2025                    852309.1   
                  2026                    886910.9   
                  2027                    921429.0   
                  2028                    956057.1   
             RTS  2025                    842998.2   
                  2026                    877451.8   
                  2027                    912175.4   
                  2028                    947299.6   
             RTX  2025                    836813.0   
                  2026                    871263.3   
                  2027                    906832.8   
                  2028                    941245.5   
             UCC  2025                 -11916817.9   
                  2026                 -12404254.5   
                  2027                 -12889266.5   
                  2028                 -13383992.2   
             UFM  2025                   -352171.5   
                  2026                   -366651.5   
                  2027                   -381275.4   
                  2028                   -395940.5   
             UFS  2025                   -159363.0   
                  2026                   -165428.0   
                  2027                   -171615.3   
                  2028                   -178151.3   
             URL  2025                   -997354.4   
                  2026                  -1038151.9   
                  2027                  -1078742.2   
                  2028                  -1120146.9   
             URM  2025                   -264475.1   
                  2026                   -274803.7   
                  2027                   -285516.9   
                  2028                   -296350.4   
             URS  2025                    252815.3   
                  2026                    264785.8   
                  2027                    274673.8   
                  2028                    285395.4   
             URV  2025                  -3655998.5   
                  2026                  -3805542.5   
                  2027                  -3954339.9   
                  2028                  -4106117.3   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Queensland   RAM  2025                                 297.2      4531.0  
                  2026                                 304.8      4599.0  
                  2027                                 313.9      4660.0  
                  2028                                 323.6      4722.0  
             RAV  2025                                  61.8    106326.0  
                  2026                                  63.4    107917.0  
                  2027                                  65.1    109355.0  
                  2028                                  66.6    110812.0  
             RTL  2025                                 218.2     47926.0  
                  2026                                 223.6     48641.0  
                  2027                                 228.8     49289.0  
                  2028                                 234.6     49946.0  
             RTM  2025                                 47

### South Australia

In [74]:
display_data(sim_grants_20_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    210515.5   
                     2026                    217316.6   
                     2027                    224475.6   
                     2028                    232198.4   
                RAM  2025                    161695.8   
                     2026                    167047.1   
                     2027                    172795.3   
                     2028                    178504.7   
                RAS  2025                    133754.9   
                     2026                    138217.7   
                     2027                    142610.9   
                     2028                    147135.4   
                RAV  2025                    287750.6   
                     2026                    297663.0   
                     2027                    306259.4   
                     2028                    316324.3   
                RSG  2025                    -40689.1   
                     2026                    -42037.5   
                     2027                    -43353.1   
                     2028                    -44720.4   
                UCC  2025                   -248554.8   
                     2026                   -256716.1   
                     2027                   -264880.2   
                     2028                   -273338.2   
                UDL  2025                   -965677.9   
                     2026                   -997390.9   
                     2027                  -1029122.3   
                     2028                  -1061906.8   
                UDM  2025                   -446127.0   
                     2026                   -460771.7   
                     2027                   -475410.8   
                     2028                   -490605.1   
                UDS  2025                   -171137.1   
                     2026                   -176751.2   
                     2027                   -182369.8   
                     2028                   -188197.0   
                UDV  2025                   -790490.4   
                     2026                   -816364.8   
                     2027                   -839556.0   
                     2028                   -864089.7   
                UFL  2025                    962377.7   
                     2026                    998968.9   
                     2027                   1031565.2   
                     2028                   1047561.3   
                UFM  2025                   -372754.1   
                     2026                   -384985.2   
                     2027                   -397224.6   
                     2028                   -409918.7   
                UFS  2025                     17417.8   
                     2026                     18527.3   
                     2027                     19441.5   
                     2028                     21160.2   
                UFV  2025                   -893410.2   
                     2026                   -928913.1   
                     2027                   -959893.4   
                     2028                   -987693.6   
                URM  2025                   -232497.7   
                     2026                   -240556.8   
                     2027                   -248017.8   
                     2028                   -255801.2   
                URS  2025                    267542.9   
                     2026                    275781.7   
                     2027                    283573.6   
                     2028                    293057.3   

                           Expected Change in GPG Per Capita($)  Population  
Jurisdiction    ACLG Year                                                    
South Australia RAL  2025                                  24.0     85943.0  
                     2026     

### Western Australia

In [75]:
display_data(sim_grants_20_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    199458.2   
                       2026                    208049.1   
                       2027                    216175.1   
                       2028                    224746.7   
                  RAM  2025                    108948.6   
                       2026                    113771.4   
                       2027                    118555.8   
                       2028                    123545.4   
                  RAS  2025                    154843.1   
                       2026                    161698.2   
                       2027                    168380.8   
                       2028                    174962.4   
                  RAV  2025                    204658.1   
                       2026                    213304.1   
                       2027                    222204.5   
                       2028                    231011.8   
                  RSG  2025                    -94458.4   
                       2026                    -98547.8   
                       2027                   -102610.5   
                       2028                   -106773.3   
                  RTL  2025                    204608.1   
                       2026                    213631.4   
                       2027                    221412.3   
                       2028                    230621.3   
                  RTM  2025                    272879.9   
                       2026                    284826.5   
                       2027                    296067.9   
                       2028                    307356.8   
                  RTS  2025                    241265.8   
                       2026                    251983.0   
                       2027                    262557.6   
                       2028                    271782.0   
                  RTX  2025                    366189.0   
                       2026                    383082.4   
                       2027                    399253.8   
                       2028                    415591.6   
                  UCC  2025                   -297977.0   
                       2026                   -310975.2   
                       2027                   -323516.2   
                       2028                   -336286.3   
                  UDL  2025                  -1051189.5   
                       2026                  -1097050.7   
                       2027                  -1141301.3   
                       2028                  -1186359.9   
                  UDM  2025                   -474622.5   
                       2026                   -495329.9   
                       2027                   -515307.7   
                       2028                   -535653.0   
                  UDS  2025                   -221345.2   
                       2026                   -231000.5   
                       2027                   -240318.1   
                       2028                   -249807.3   
                  UDV  2025                  -1689711.4   
                       2026                  -1764102.8   
                       2027                  -1835372.0   
                       2028                  -1907968.2   
                  UFL  2025                   -900303.8   
                       2026                   -939583.7   
                       2027                   -977479.9   
                       2028                  -1016073.2   
                  UFM  2025                   -421266.9   
                       2026                   -439801.6   
                       2027                   -458199.4   
                       2028                   -476707.3   
                  UFS  2025                    -79085.4   
                       2026                    -83178.5   
                 

### Tasmania

In [76]:
display_data(sim_grants_20_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    139641.4   
                  2026                    143785.1   
                  2027                    147592.4   
                  2028                    151978.4   
             RAM  2025                     54965.7   
                  2026                     56786.0   
                  2027                     58332.6   
                  2028                     60147.9   
             RAS  2025                    126274.6   
                  2026                    129330.6   
                  2027                    132839.6   
                  2028                    136541.1   
             RAV  2025                     80271.8   
                  2026                     82330.7   
                  2027                     84492.2   
                  2028                     86922.7   
             UCC  2025                   -464209.7   
                  2026                   -477157.2   
                  2027                   -490104.6   
                  2028                   -504900.8   
             UFM  2025                   -443872.6   
                  2026                   -456249.9   
                  2027                   -468629.1   
                  2028                   -482773.9   
             UFS  2025                     56021.0   
                  2026                     57311.9   
                  2027                     59542.6   
                  2028                     61201.0   
             URM  2025                   -586554.6   
                  2026                   -602916.5   
                  2027                   -619269.5   
                  2028                   -637967.4   
             URS  2025                     85352.3   
                  2026                     87514.0   
                  2027                     89234.6   
                  2028                     92820.9   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Tasmania     RAL  2025                                  17.8     48472.0  
                  2026                                  18.3     48641.0  
                  2027                                  18.7     48810.0  
                  2028                                  19.2     48978.0  
             RAM  2025                                  10.4     27678.0  
                  2026                                  10.7     27773.0  
                  2027                                  11.0     27869.0  
                  2028                                  11.3     27966.0  
             RAS  2025                                  82.5      3122.0  
                  2026                                  84.3      3133.0  
                  2027                                  86.3      3144.0  
                  2028                                  88.4      3154.0  
             RAV  2025                                   4.9    113763.0  
                  2026                                   5.0    114158.0  
                  2027                                   5.2    114553.0  
                  2028                                   5.3    114948.0  
             UCC  2025                                  -8.7     53264.0  
                  2026                                  -8.9     53449.0  
                  2027                                  -9.1     53634.0  
                  2028                                  -9.4     53819.0  
             UFM  2025                                  -8.7    149166.0  
                  2026                                  -8.9    149683.0  
                  2027                                  -9.1    150201.0  
                  2028                                  -9.4    150718.0  
             UFS  202

### Northern Territory

In [77]:
display_data(sim_grants_20_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    256061.9   
                        2026                    263995.9   
                        2027                    272140.7   
                        2028                    281181.6   
                   RTM  2025                     19835.9   
                        2026                     20342.6   
                        2027                     20964.7   
                        2028                     21742.5   
                   RTS  2025                     -4234.4   
                        2026                     -4369.1   
                        2027                     -4503.0   
                        2028                     -4656.9   
                   RTX  2025                      2449.5   
                        2026                      2532.5   
                        2027                      2604.2   
                        2028                      2710.3   
                   UCC  2025                   -779386.3   
                        2026                   -803857.5   
                        2027                   -828324.1   
                        2028                   -856281.1   
                   UFS  2025                   -213532.5   
                        2026                   -220232.5   
                        2027                   -226936.6   
                        2028                   -234600.6   
                   URM  2025                   -374924.9   
                        2026                   -386694.5   
                        2027                   -398460.8   
                        2028                   -411907.0   
                   URS  2025                   -214442.3   
                        2026                   -221132.5   
                        2027                   -227870.4   
                        2028                   -235531.5   

                              Expected Change in GPG Per Capita($)  Population  
Jurisdiction       ACLG Year                                                    
Northern Territory RTL  2025                                  32.3     52234.0  
                        2026                                  33.0     52640.0  
                        2027                                  33.9     52843.0  
                        2028                                  34.8     53250.0  
                   RTM  2025                                   5.4      4380.0  
                        2026                                   5.5      4415.0  
                        2027                                   5.7      4432.0  
                        2028                                   5.8      4466.0  
                   RTS  2025                                  -8.7       488.0  
                        2026                                  -8.9       492.0  
                        2027                                  -9.1       494.0  
                        2028                                  -9.4       498.0  
                   RTX  2025                                  14.0       175.0  
                        2026                                  14.4       176.0  
                        2027                                  14.7       177.0  
                        2028                                  15.2       178.0  
                   UCC  2025                                  -8.7     89822.0  
                        2026                                  -8.9     90521.0  
                        2027                                  -9.1     90871.0  
                        2028                                  -9.4     91570.0  
                   UFS  2025                                  -8.7     24609.0  
                        2026                                  -8.9     24800.0  
                        2027                 

## A 10% Per Capita Minimum Grant

### New South Wales

In [78]:
display_data(sim_grants_10_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                   1118013.5   
                     2026                   1159624.6   
                     2027                   1201034.7   
                     2028                   1242592.9   
                RAM  2025                   1048556.5   
                     2026                   1087423.1   
                     2027                   1126024.9   
                     2028                   1165925.5   
                RAS  2025                   1456087.9   
                     2026                   1507172.3   
                     2027                   1560424.8   
                     2028                   1614323.3   
                RAV  2025                   -751099.3   
                     2026                   -778767.4   
                     2027                   -807638.7   
                     2028                   -836978.0   
                RTL  2025                   1648963.1   
                     2026                   1715584.5   
                     2027                   1788563.4   
                     2028                   1852990.6   
                RTM  2025                   1831861.6   
                     2026                   1894703.0   
                     2027                   1968591.0   
                     2028                   2037047.9   
                RTX  2025                     77178.1   
                     2026                     80414.0   
                     2027                     83143.1   
                     2028                     86533.7   
                UCC  2025                  -4132583.7   
                     2026                  -4286054.9   
                     2027                  -4442291.5   
                     2028                  -4602407.2   
                UDL  2025                  -1412741.2   
                     2026                  -1465207.8   
                     2027                  -1518619.9   
                     2028                  -1573352.7   
                UDM  2025                   -781310.4   
                     2026                   -810364.8   
                     2027                   -839918.1   
                     2028                   -870282.0   
                UDS  2025                   -226142.4   
                     2026                   -234498.5   
                     2027                   -243050.9   
                     2028                   -251832.4   
                UDV  2025                  -4392429.4   
                     2026                  -4554479.4   
                     2027                  -4719993.7   
                     2028                  -4889778.7   
                UFL  2025                  -1266962.8   
                     2026                  -1316330.2   
                     2027                  -1359854.9   
                     2028                  -1403510.6   
                UFM  2025                   -736861.4   
                     2026                   -762908.8   
                     2027                   -789700.7   
                     2028                   -816762.0   
                UFV  2025                  -2374097.0   
                     2026                  -2465722.8   
                     2027                  -2554935.7   
                     2028                  -2652747.5   
                URL  2025                    277352.3   
                     2026                    289808.9   
                     2027                    300784.6   
                     2028                    310780.2   
                URM  2025                    530758.6   
                     2026                    549280.1   
                     2027                    570317.9   
                     2028                    592803.5   
                URS  2025            

### Victoria

In [79]:
display_data(sim_grants_10_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                    841000.5   
                  2026                    876444.6   
                  2027                    911969.1   
                  2028                    947524.1   
             RAM  2025                    945886.0   
                  2026                    982275.0   
                  2027                   1023913.5   
                  2028                   1065195.5   
             RAV  2025                    799962.2   
                  2026                    834258.5   
                  2027                    868565.5   
                  2028                    901049.5   
             UCC  2025                  -3198422.1   
                  2026                  -3331167.4   
                  2027                  -3465474.7   
                  2028                  -3602904.9   
             UDL  2025                  -1649186.6   
                  2026                  -1717948.1   
                  2027                  -1787296.8   
                  2028                  -1857485.3   
             UDV  2025                  -1959444.5   
                  2026                  -2041596.9   
                  2027                  -2131937.6   
                  2028                  -2213736.9   
             UFL  2025                    622022.4   
                  2026                    658386.7   
                  2027                    696383.0   
                  2028                    739145.7   
             UFM  2025                     -5307.4   
                  2026                     -5409.7   
                  2027                     -4462.8   
                  2028                     -3865.3   
             UFS  2025                    604416.5   
                  2026                    629195.5   
                  2027                    660123.2   
                  2028                    684946.1   
             UFV  2025                   -206881.6   
                  2026                   -215689.8   
                  2027                   -225632.5   
                  2028                   -237282.7   
             URL  2025                   1593621.5   
                  2026                   1668050.4   
                  2027                   1739711.9   
                  2028                   1799623.0   
             URM  2025                   1343353.9   
                  2026                   1397453.3   
                  2027                   1457230.7   
                  2028                   1516839.5   
             URS  2025                   1127710.7   
                  2026                   1174015.0   
                  2027                   1221713.8   
                  2028                   1268509.0   
             URV  2025                    323539.5   
                  2026                    326563.9   
                  2027                    343374.7   
                  2028                    377510.4   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Victoria     RAL  2025                                 114.4     52851.0  
                  2026                                 117.3     53684.0  
                  2027                                 120.3     54451.0  
                  2028                                 123.2     55232.0  
             RAM  2025                                 230.6      4102.0  
                  2026                                 235.7      4167.0  
                  2027                                 242.2      4227.0  
                  2028                                 248.5      4287.0  
             RAV  2025                                  49.4    258681.0  
                  2026                                  50.7    26

### Queensland

In [80]:
display_data(sim_grants_10_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   2683521.5   
                  2026                   2787894.9   
                  2027                   2910546.2   
                  2028                   3018041.1   
             RAV  2025                   1665645.9   
                  2026                   1734004.1   
                  2027                   1803724.6   
                  2028                   1870938.5   
             RTL  2025                   2324781.0   
                  2026                   2422055.8   
                  2027                   2520646.8   
                  2028                   2620835.8   
             RTM  2025                   1706852.6   
                  2026                   1776425.5   
                  2027                   1843504.5   
                  2028                   1917815.7   
             RTS  2025                   1684086.6   
                  2026                   1754108.1   
                  2027                   1817872.8   
                  2028                   1884957.9   
             RTX  2025                   1670360.0   
                  2026                   1737205.1   
                  2027                   1806080.8   
                  2028                   1877143.0   
             UCC  2025                 -23833635.7   
                  2026                 -24808509.0   
                  2027                 -25778533.0   
                  2028                 -26767984.5   
             UFM  2025                   -704491.2   
                  2026                   -733137.6   
                  2027                   -761798.5   
                  2028                   -790963.2   
             UFS  2025                   -318124.2   
                  2026                   -331290.6   
                  2027                   -344778.9   
                  2028                   -358089.2   
             URL  2025                  -1994708.8   
                  2026                  -2076303.9   
                  2027                  -2157484.4   
                  2028                  -2240293.7   
             URM  2025                   -528414.0   
                  2026                   -549503.8   
                  2027                   -571344.2   
                  2028                   -593904.6   
             URS  2025                    505373.9   
                  2026                    524899.1   
                  2027                    545904.8   
                  2028                    564220.3   
             URV  2025                  -7311997.0   
                  2026                  -7611084.9   
                  2027                  -7908679.9   
                  2028                  -8212234.6   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Queensland   RAM  2025                                 592.3      4531.0  
                  2026                                 606.2      4599.0  
                  2027                                 624.6      4660.0  
                  2028                                 639.1      4722.0  
             RAV  2025                                 123.9    106326.0  
                  2026                                 127.0    107917.0  
                  2027                                 130.4    109355.0  
                  2028                                 133.5    110812.0  
             RTL  2025                                 436.2     47926.0  
                  2026                                 447.4     48641.0  
                  2027                                 459.6     49289.0  
                  2028                                 471.2     49946.0  
             RTM  2025                                 94

### South Australia

In [81]:
display_data(sim_grants_10_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    420653.1   
                     2026                    434451.4   
                     2027                    448328.2   
                     2028                    462480.1   
                RAM  2025                    323800.1   
                     2026                    335035.8   
                     2027                    344099.1   
                     2028                    355744.0   
                RAS  2025                    267351.9   
                     2026                    276528.8   
                     2027                    285343.1   
                     2028                    294874.6   
                RAV  2025                    572848.3   
                     2026                    589873.0   
                     2027                    607550.8   
                     2028                    627441.9   
                RSG  2025                    -81191.2   
                     2026                    -83938.1   
                     2027                    -86636.6   
                     2028                    -89281.5   
                UCC  2025                   -497109.5   
                     2026                   -513432.1   
                     2027                   -529760.3   
                     2028                   -546676.4   
                UDL  2025                  -1931395.6   
                     2026                  -1994734.3   
                     2027                  -2058197.4   
                     2028                  -2124054.4   
                UDM  2025                   -892467.7   
                     2026                   -921564.1   
                     2027                   -950837.8   
                     2028                   -981166.3   
                UDS  2025                   -342274.3   
                     2026                   -353502.5   
                     2027                   -364739.5   
                     2028                   -376394.0   
                UDV  2025                  -1578648.7   
                     2026                  -1630135.5   
                     2027                  -1677691.6   
                     2028                  -1733913.8   
                UFL  2025                   1942574.0   
                     2026                   2002834.3   
                     2027                   2076519.5   
                     2028                   2158802.6   
                UFM  2025                   -745508.1   
                     2026                   -769970.4   
                     2027                   -794449.3   
                     2028                   -819837.3   
                UFS  2025                     31459.8   
                     2026                     30560.1   
                     2027                     33467.8   
                     2028                     33264.2   
                UFV  2025                  -1790176.7   
                     2026                  -1839911.7   
                     2027                  -1893635.1   
                     2028                  -1960261.3   
                URM  2025                   -465575.1   
                     2026                   -482738.6   
                     2027                   -494670.6   
                     2028                   -510535.0   
                URS  2025                    536801.7   
                     2026                    553387.7   
                     2027                    568743.5   
                     2028                    584163.5   

                           Expected Change in GPG Per Capita($)  Population  
Jurisdiction    ACLG Year                                                    
South Australia RAL  2025                                  48.0     85943.0  
                     2026     

### Western Australia

In [82]:
display_data(sim_grants_10_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    398222.3   
                       2026                    415020.0   
                       2027                    432085.8   
                       2028                    449870.3   
                  RAM  2025                    218399.4   
                       2026                    228229.5   
                       2027                    237121.8   
                       2028                    246571.7   
                  RAS  2025                    309919.2   
                       2026                    323839.1   
                       2027                    336993.4   
                       2028                    350249.2   
                  RAV  2025                    409631.3   
                       2026                    424860.2   
                       2027                    442776.8   
                       2028                    458782.1   
                  RSG  2025                   -189542.7   
                       2026                   -196276.8   
                       2027                   -204300.3   
                       2028                   -213299.7   
                  RTL  2025                    407866.4   
                       2026                    425931.4   
                       2027                    442772.7   
                       2028                    459546.7   
                  RTM  2025                    545284.0   
                       2026                    567747.6   
                       2027                    590924.3   
                       2028                    615226.9   
                  RTS  2025                    483059.1   
                       2026                    504889.6   
                       2027                    525759.9   
                       2028                    546570.8   
                  RTX  2025                    731015.0   
                       2026                    761658.9   
                       2027                    791438.7   
                       2028                    820627.4   
                  UCC  2025                   -595953.9   
                       2026                   -621950.4   
                       2027                   -647032.4   
                       2028                   -672572.6   
                  UDL  2025                  -2102378.9   
                       2026                  -2194101.4   
                       2027                  -2282602.6   
                       2028                  -2372719.8   
                  UDM  2025                   -949245.1   
                       2026                   -990659.7   
                       2027                  -1030615.5   
                       2028                  -1071306.1   
                  UDS  2025                   -442690.4   
                       2026                   -462001.0   
                       2027                   -480636.1   
                       2028                   -499614.6   
                  UDV  2025                  -3379147.2   
                       2026                  -3527046.3   
                       2027                  -3669724.7   
                       2028                  -3814052.9   
                  UFL  2025                  -1800607.7   
                       2026                  -1879167.4   
                       2027                  -1954959.8   
                       2028                  -2032146.4   
                  UFM  2025                   -841219.9   
                       2026                   -879118.0   
                       2027                   -914887.2   
                       2028                   -952357.7   
                  UFS  2025                   -159427.0   
                       2026                   -167037.2   
                 

### Tasmania

In [83]:
display_data(sim_grants_10_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    279646.2   
                  2026                    287422.2   
                  2027                    295044.5   
                  2028                    303736.4   
             RAM  2025                    109322.9   
                  2026                    112731.4   
                  2027                    115081.1   
                  2028                    119447.0   
             RAS  2025                    252974.2   
                  2026                    259507.2   
                  2027                    267456.9   
                  2028                    275811.9   
             RAV  2025                    158791.7   
                  2026                    163016.4   
                  2027                    167100.3   
                  2028                    172227.8   
             UCC  2025                   -928419.5   
                  2026                   -954314.3   
                  2027                   -980209.3   
                  2028                  -1009801.5   
             UFM  2025                   -887745.1   
                  2026                   -912499.8   
                  2027                   -937258.1   
                  2028                   -965547.8   
             UFS  2025                    114748.3   
                  2026                    117271.7   
                  2027                    121190.7   
                  2028                    125721.7   
             URM  2025                  -1173109.2   
                  2026                  -1205833.1   
                  2027                  -1238539.0   
                  2028                  -1275934.8   
             URS  2025                    171655.0   
                  2026                    177306.0   
                  2027                    182999.7   
                  2028                    187034.2   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Tasmania     RAL  2025                                  35.7     48472.0  
                  2026                                  36.5     48641.0  
                  2027                                  37.4     48810.0  
                  2028                                  38.3     48978.0  
             RAM  2025                                  20.8     27678.0  
                  2026                                  21.4     27773.0  
                  2027                                  21.8     27869.0  
                  2028                                  22.5     27966.0  
             RAS  2025                                 165.5      3122.0  
                  2026                                 169.1      3133.0  
                  2027                                 173.7      3144.0  
                  2028                                 178.5      3154.0  
             RAV  2025                                   9.8    113763.0  
                  2026                                  10.0    114158.0  
                  2027                                  10.2    114553.0  
                  2028                                  10.5    114948.0  
             UCC  2025                                 -17.4     53264.0  
                  2026                                 -17.9     53449.0  
                  2027                                 -18.3     53634.0  
                  2028                                 -18.8     53819.0  
             UFM  2025                                 -17.4    149166.0  
                  2026                                 -17.9    149683.0  
                  2027                                 -18.3    150201.0  
                  2028                                 -18.8    150718.0  
             UFS  202

### Northern Territory

In [84]:
display_data(sim_grants_10_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    512590.1   
                        2026                    528928.1   
                        2027                    544976.5   
                        2028                    563664.2   
                   RTM  2025                     39734.3   
                        2026                     40406.1   
                        2027                     41988.3   
                        2028                     43359.5   
                   RTS  2025                     -8468.8   
                        2026                     -8738.3   
                        2027                     -9006.0   
                        2028                     -9313.7   
                   RTX  2025                      4829.8   
                        2026                      4949.0   
                        2027                      5092.4   
                        2028                      5271.6   
                   UCC  2025                  -1558772.6   
                        2026                  -1607715.1   
                        2027                  -1656648.2   
                        2028                  -1712562.2   
                   UFS  2025                   -427065.0   
                        2026                   -440465.0   
                        2027                   -453873.2   
                        2028                   -469201.3   
                   URM  2025                   -749849.8   
                        2026                   -773389.1   
                        2027                   -796921.6   
                        2028                   -823814.0   
                   URS  2025                   -428878.6   
                        2026                   -442346.3   
                        2027                   -455823.3   
                        2028                   -471217.5   

                              Expected Change in GPG Per Capita($)  Population  
Jurisdiction       ACLG Year                                                    
Northern Territory RTL  2025                                  64.6     52234.0  
                        2026                                  66.1     52640.0  
                        2027                                  67.8     52843.0  
                        2028                                  69.6     53250.0  
                   RTM  2025                                  10.9      4380.0  
                        2026                                  11.0      4415.0  
                        2027                                  11.4      4432.0  
                        2028                                  11.6      4466.0  
                   RTS  2025                                 -17.4       488.0  
                        2026                                 -17.8       492.0  
                        2027                                 -18.2       494.0  
                        2028                                 -18.7       498.0  
                   RTX  2025                                  27.6       175.0  
                        2026                                  28.1       176.0  
                        2027                                  28.8       177.0  
                        2028                                  29.6       178.0  
                   UCC  2025                                 -17.4     89822.0  
                        2026                                 -17.8     90521.0  
                        2027                                 -18.2     90871.0  
                        2028                                 -18.7     91570.0  
                   UFS  2025                                 -17.4     24609.0  
                        2026                                 -17.8     24800.0  
                        2027                 

## A World With No Per Capita Minimum Grant

### New South Wales

In [85]:
display_data(sim_grants_0_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                   1675710.8   
                     2026                   1737673.1   
                     2027                   1802265.1   
                     2028                   1868082.2   
                RAM  2025                   1573049.5   
                     2026                   1629947.9   
                     2027                   1689556.7   
                     2028                   1753135.1   
                RAS  2025                   2168184.5   
                     2026                   2253240.8   
                     2027                   2333240.8   
                     2028                   2417164.5   
                RAV  2025                  -1127693.8   
                     2026                  -1168428.6   
                     2027                  -1210498.1   
                     2028                  -1255892.4   
                RTL  2025                   2484634.0   
                     2026                   2557939.8   
                     2027                   2641500.9   
                     2028                   2729332.2   
                RTM  2025                   2769161.6   
                     2026                   2861365.4   
                     2027                   2963972.2   
                     2028                   3079411.4   
                RTX  2025                    115781.6   
                     2026                    120081.0   
                     2027                    124844.9   
                     2028                    129535.5   
                UCC  2025                  -6198875.5   
                     2026                  -6429082.4   
                     2027                  -6663437.3   
                     2028                  -6903610.8   
                UDL  2025                  -2119111.7   
                     2026                  -2197811.6   
                     2027                  -2277929.9   
                     2028                  -2360029.1   
                UDM  2025                  -1171912.9   
                     2026                  -1215269.1   
                     2027                  -1259553.0   
                     2028                  -1305048.7   
                UDS  2025                   -339246.8   
                     2026                   -351882.8   
                     2027                   -364771.4   
                     2028                   -377683.8   
                UDV  2025                  -6588948.9   
                     2026                  -6834205.7   
                     2027                  -7085153.1   
                     2028                  -7340161.0   
                UFL  2025                  -1902095.4   
                     2026                  -1968681.2   
                     2027                  -2036046.5   
                     2028                  -2109378.6   
                UFM  2025                  -1104914.0   
                     2026                  -1144663.4   
                     2027                  -1186909.8   
                     2028                  -1228150.8   
                UFV  2025                  -3573243.8   
                     2026                  -3711826.3   
                     2027                  -3852556.8   
                     2028                  -3993785.8   
                URL  2025                    417957.4   
                     2026                    429540.7   
                     2027                    441012.5   
                     2028                    456132.9   
                URM  2025                    800840.4   
                     2026                    832401.5   
                     2027                    863637.7   
                     2028                    897063.3   
                URS  2025            

### Victoria

In [86]:
display_data(sim_grants_0_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                   1265205.2   
                  2026                   1316643.9   
                  2027                   1372333.0   
                  2028                   1424367.1   
             RAM  2025                   1420094.6   
                  2026                   1482023.0   
                  2027                   1535457.6   
                  2028                   1597256.4   
             RAV  2025                   1202128.5   
                  2026                   1252240.4   
                  2027                   1305767.8   
                  2028                   1357366.3   
             UCC  2025                  -4797633.2   
                  2026                  -4996751.1   
                  2027                  -5198212.1   
                  2028                  -5404357.4   
             UDL  2025                  -2474051.3   
                  2026                  -2578574.4   
                  2027                  -2683111.7   
                  2028                  -2789204.8   
             UDV  2025                  -2939746.7   
                  2026                  -3066183.4   
                  2027                  -3184423.0   
                  2028                  -3312011.7   
             UFL  2025                    922989.0   
                  2026                    986973.0   
                  2027                   1013275.0   
                  2028                   1069269.4   
             UFM  2025                     -8259.9   
                  2026                     -7080.2   
                  2027                     -6006.7   
                  2028                    -11226.8   
             UFS  2025                    909100.2   
                  2026                    951045.2   
                  2027                    989544.8   
                  2028                   1023971.3   
             UFV  2025                   -311637.2   
                  2026                   -329407.8   
                  2027                   -345298.8   
                  2028                   -345757.3   
             URL  2025                   2368925.1   
                  2026                   2467575.2   
                  2027                   2561546.2   
                  2028                   2671735.5   
             URM  2025                   2012098.2   
                  2026                   2097943.8   
                  2027                   2181869.7   
                  2028                   2263353.7   
             URS  2025                   1697418.6   
                  2026                   1768837.3   
                  2027                   1838929.9   
                  2028                   1913284.4   
             URV  2025                    511977.8   
                  2026                    547415.7   
                  2027                    547834.4   
                  2028                    597273.3   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Victoria     RAL  2025                                 172.0     52851.0  
                  2026                                 176.2     53684.0  
                  2027                                 181.1     54451.0  
                  2028                                 185.4     55232.0  
             RAM  2025                                 346.2      4102.0  
                  2026                                 355.7      4167.0  
                  2027                                 363.2      4227.0  
                  2028                                 372.6      4287.0  
             RAV  2025                                  74.3    258681.0  
                  2026                                  76.2    26

### Queensland

In [87]:
display_data(sim_grants_0_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   4069703.1   
                  2026                   4225680.6   
                  2027                   4379033.2   
                  2028                   4542561.9   
             RAV  2025                   2494902.9   
                  2026                   2595995.5   
                  2027                   2697615.4   
                  2028                   2800747.1   
             RTL  2025                   3490343.5   
                  2026                   3634269.6   
                  2027                   3771415.5   
                  2028                   3923882.7   
             RTM  2025                   2558601.9   
                  2026                   2665396.4   
                  2027                   2768367.3   
                  2028                   2873218.9   
             RTS  2025                   2520897.6   
                  2026                   2622874.9   
                  2027                   2727140.1   
                  2028                   2830035.6   
             RTX  2025                   2502495.6   
                  2026                   2601072.8   
                  2027                   2705245.1   
                  2028                   2809025.3   
             UCC  2025                 -35750453.6   
                  2026                 -37212763.6   
                  2027                 -38667799.5   
                  2028                 -40151976.7   
             UFM  2025                  -1057324.5   
                  2026                  -1100578.5   
                  2027                  -1143596.3   
                  2028                  -1187595.9   
             UFS  2025                   -477285.9   
                  2026                   -496493.0   
                  2027                   -515479.7   
                  2028                   -534969.8   
             URL  2025                  -2992063.2   
                  2026                  -3114455.8   
                  2027                  -3236226.6   
                  2028                  -3360440.6   
             URM  2025                   -794035.9   
                  2026                   -826446.9   
                  2027                   -857542.7   
                  2028                   -889039.3   
             URS  2025                    759746.8   
                  2026                    793079.7   
                  2027                    825914.5   
                  2028                    856806.9   
             URV  2025                 -10967995.6   
                  2026                 -11416627.4   
                  2027                 -11863019.8   
                  2028                 -12318351.9   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Queensland   RAM  2025                                 898.2      4531.0  
                  2026                                 918.8      4599.0  
                  2027                                 939.7      4660.0  
                  2028                                 962.0      4722.0  
             RAV  2025                                 185.6    106326.0  
                  2026                                 190.2    107917.0  
                  2027                                 195.1    109355.0  
                  2028                                 199.9    110812.0  
             RTL  2025                                 655.4     47926.0  
                  2026                                 672.7     48641.0  
                  2027                                 689.5     49289.0  
                  2028                                 707.2     49946.0  
             RTM  2025                                142

### South Australia

In [88]:
display_data(sim_grants_0_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    631126.5   
                     2026                    652919.7   
                     2027                    673267.6   
                     2028                    693830.9   
                RAM  2025                    484543.6   
                     2026                    502006.0   
                     2027                    518329.1   
                     2028                    531948.9   
                RAS  2025                    401936.2   
                     2026                    415129.6   
                     2027                    428317.1   
                     2028                    441362.5   
                RAV  2025                    858795.6   
                     2026                    889396.6   
                     2027                    921424.0   
                     2028                    950293.3   
                RSG  2025                   -121853.1   
                     2026                   -125848.2   
                     2027                   -129915.9   
                     2028                   -134004.7   
                UCC  2025                   -745664.3   
                     2026                   -770148.2   
                     2027                   -794640.5   
                     2028                   -820014.7   
                UDL  2025                  -2897147.9   
                     2026                  -2992217.8   
                     2027                  -3087666.6   
                     2028                  -3186134.0   
                UDM  2025                  -1338509.0   
                     2026                  -1382741.8   
                     2027                  -1426585.5   
                     2028                  -1472271.6   
                UDS  2025                   -513411.4   
                     2026                   -530253.7   
                     2027                   -547109.3   
                     2028                   -564591.0   
                UDV  2025                  -2373134.8   
                     2026                  -2459027.8   
                     2027                  -2533338.8   
                     2028                  -2602069.0   
                UFL  2025                   2890148.9   
                     2026                   3013063.4   
                     2027                   3085849.2   
                     2028                   3205215.2   
                UFM  2025                  -1118262.2   
                     2026                  -1154955.7   
                     2027                  -1191673.9   
                     2028                  -1229756.0   
                UFS  2025                     52520.5   
                     2026                     55692.2   
                     2027                     59267.8   
                     2028                     59801.1   
                UFV  2025                  -2677200.3   
                     2026                  -2773272.4   
                     2027                  -2860583.0   
                     2028                  -2942991.6   
                URM  2025                   -695910.5   
                     2026                   -718484.1   
                     2027                   -739829.3   
                     2028                   -764560.3   
                URS  2025                    806924.7   
                     2026                    829191.7   
                     2027                    853723.5   
                     2028                    880805.3   

                           Expected Change in GPG Per Capita($)  Population  
Jurisdiction    ACLG Year                                                    
South Australia RAL  2025                                  72.0     85943.0  
                     2026     

### Western Australia

In [89]:
display_data(sim_grants_0_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    597346.3   
                       2026                    622877.4   
                       2027                    646916.7   
                       2028                    672424.2   
                  RAM  2025                    327438.3   
                       2026                    342101.0   
                       2027                    356083.6   
                       2028                    370962.3   
                  RAS  2025                    464986.3   
                       2026                    485037.5   
                       2027                    504691.3   
                       2028                    524279.2   
                  RAV  2025                    612288.3   
                       2026                    642318.3   
                       2027                    666360.8   
                       2028                    692665.8   
                  RSG  2025                   -283526.8   
                       2026                   -295952.0   
                       2027                   -307308.7   
                       2028                   -319230.7   
                  RTL  2025                    612928.1   
                       2026                    639938.9   
                       2027                    665865.3   
                       2028                    689188.1   
                  RTM  2025                    821409.6   
                       2026                    856943.8   
                       2027                    890139.8   
                       2028                    927583.9   
                  RTS  2025                    723314.9   
                       2026                    754089.3   
                       2027                    784680.3   
                       2028                    815546.1   
                  RTX  2025                   1097158.9   
                       2026                   1147075.0   
                       2027                   1191161.1   
                       2028                   1241109.3   
                  UCC  2025                   -893930.9   
                       2026                   -932925.6   
                       2027                   -970548.6   
                       2028                  -1008859.0   
                  UDL  2025                  -3153568.4   
                       2026                  -3291152.1   
                       2027                  -3423904.0   
                       2028                  -3559079.8   
                  UDM  2025                  -1423867.6   
                       2026                  -1485989.6   
                       2027                  -1545923.2   
                       2028                  -1606959.1   
                  UDS  2025                   -664035.6   
                       2026                   -693001.5   
                       2027                   -720954.2   
                       2028                   -749421.8   
                  UDV  2025                  -5069241.3   
                       2026                  -5290015.1   
                       2027                  -5502249.6   
                       2028                  -5719992.9   
                  UFL  2025                  -2700911.5   
                       2026                  -2818751.1   
                       2027                  -2932439.7   
                       2028                  -3048219.6   
                  UFM  2025                  -1266912.5   
                       2026                  -1323300.2   
                       2027                  -1375367.3   
                       2028                  -1428875.4   
                  UFS  2025                   -242139.2   
                       2026                   -252517.3   
                 

### Tasmania

In [90]:
display_data(sim_grants_0_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    419800.0   
                  2026                    432485.6   
                  2027                    442990.6   
                  2028                    456723.7   
             RAM  2025                    164676.2   
                  2026                    168837.3   
                  2027                    175002.2   
                  2028                    180023.6   
             RAS  2025                    381013.9   
                  2026                    390211.5   
                  2027                    401179.1   
                  2028                    413429.8   
             RAV  2025                    241042.9   
                  2026                    248093.0   
                  2027                    255684.7   
                  2028                    262699.6   
             UCC  2025                  -1392629.2   
                  2026                  -1431471.5   
                  2027                  -1470313.9   
                  2028                  -1514702.3   
             UFM  2025                  -1331617.7   
                  2026                  -1368749.7   
                  2027                  -1405887.2   
                  2028                  -1448321.7   
             UFS  2025                    166712.7   
                  2026                    173816.2   
                  2027                    179640.6   
                  2028                    185440.0   
             URM  2025                  -1759663.8   
                  2026                  -1808749.6   
                  2027                  -1857808.5   
                  2028                  -1913902.2   
             URS  2025                    253169.7   
                  2026                    257888.5   
                  2027                    262678.7   
                  2028                    271249.0   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Tasmania     RAL  2025                                  53.5     48472.0  
                  2026                                  55.0     48641.0  
                  2027                                  56.1     48810.0  
                  2028                                  57.7     48978.0  
             RAM  2025                                  31.3     27678.0  
                  2026                                  32.0     27773.0  
                  2027                                  33.0     27869.0  
                  2028                                  33.8     27966.0  
             RAS  2025                                 249.2      3122.0  
                  2026                                 254.5      3133.0  
                  2027                                 260.7      3144.0  
                  2028                                 267.7      3154.0  
             RAV  2025                                  14.8    113763.0  
                  2026                                  15.2    114158.0  
                  2027                                  15.6    114553.0  
                  2028                                  16.0    114948.0  
             UCC  2025                                 -26.1     53264.0  
                  2026                                 -26.8     53449.0  
                  2027                                 -27.4     53634.0  
                  2028                                 -28.1     53819.0  
             UFM  2025                                 -26.1    149166.0  
                  2026                                 -26.8    149683.0  
                  2027                                 -27.4    150201.0  
                  2028                                 -28.1    150718.0  
             UFS  202

### Northern Territory

In [91]:
display_data(sim_grants_0_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    768205.9   
                        2026                    792446.6   
                        2027                    815714.4   
                        2028                    842832.8   
                   RTM  2025                     59217.4   
                        2026                     60732.1   
                        2027                     63147.0   
                        2028                     65541.5   
                   RTS  2025                    -12703.1   
                        2026                    -13107.4   
                        2027                    -13509.0   
                        2028                    -13970.6   
                   RTX  2025                      7251.6   
                        2026                      7446.9   
                        2027                      7716.8   
                        2028                      8015.7   
                   UCC  2025                  -2338158.9   
                        2026                  -2411572.6   
                        2027                  -2484972.3   
                        2028                  -2568843.2   
                   UFS  2025                   -640597.5   
                        2026                   -660697.5   
                        2027                   -680809.8   
                        2028                   -703801.9   
                   URM  2025                  -1124774.7   
                        2026                  -1160083.6   
                        2027                  -1195382.4   
                        2028                  -1235721.0   
                   URS  2025                   -643316.2   
                        2026                   -663417.7   
                        2027                   -683529.4   
                        2028                   -706571.6   

                              Expected Change in GPG Per Capita($)  Population  
Jurisdiction       ACLG Year                                                    
Northern Territory RTL  2025                                  96.9     52234.0  
                        2026                                  99.1     52640.0  
                        2027                                 101.7     52843.0  
                        2028                                 104.4     53250.0  
                   RTM  2025                                  16.2      4380.0  
                        2026                                  16.5      4415.0  
                        2027                                  17.1      4432.0  
                        2028                                  17.6      4466.0  
                   RTS  2025                                 -26.0       488.0  
                        2026                                 -26.6       492.0  
                        2027                                 -27.3       494.0  
                        2028                                 -28.1       498.0  
                   RTX  2025                                  41.4       175.0  
                        2026                                  42.3       176.0  
                        2027                                  43.6       177.0  
                        2028                                  45.0       178.0  
                   UCC  2025                                 -26.0     89822.0  
                        2026                                 -26.6     90521.0  
                        2027                                 -27.3     90871.0  
                        2028                                 -28.1     91570.0  
                   UFS  2025                                 -26.0     24609.0  
                        2026                                 -26.6     24800.0  
                        2027                 

## And for sanity reasons, status quo

### New South Wales

In [92]:
display_data(sim_grants_30_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTX  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UCC  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URS  2025            

### Victoria

In [93]:
display_data(sim_grants_30_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UDL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UDV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Victoria     RAL  2025                                   0.0     52851.0  
                  2026                                   0.0     53684.0  
                  2027                                   0.0     54451.0  
                  2028                                   0.0     55232.0  
             RAM  2025                                   0.0      4102.0  
                  2026                                   0.0      4167.0  
                  2027                                   0.0      4227.0  
                  2028                                   0.0      4287.0  
             RAV  2025                                   0.0    258681.0  
                  2026                                   0.0    26

### Queensland

In [94]:
display_data(sim_grants_30_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTX  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Queensland   RAM  2025                                   0.0      4531.0  
                  2026                                   0.0      4599.0  
                  2027                                   0.0      4660.0  
                  2028                                   0.0      4722.0  
             RAV  2025                                   0.0    106326.0  
                  2026                                   0.0    107917.0  
                  2027                                   0.0    109355.0  
                  2028                                   0.0    110812.0  
             RTL  2025                                   0.0     47926.0  
                  2026                                   0.0     48641.0  
                  2027                                   0.0     49289.0  
                  2028                                   0.0     49946.0  
             RTM  2025                                   

### South Australia

In [95]:
display_data(sim_grants_30_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RSG  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UCC  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   

                           Expected Change in GPG Per Capita($)  Population  
Jurisdiction    ACLG Year                                                    
South Australia RAL  2025                                   0.0     85943.0  
                     2026     

### Western Australia

In [96]:
display_data(sim_grants_30_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAV  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RSG  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTX  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UCC  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDV  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFS  2025                         0.0   
                       2026                         0.0   
                 

### Tasmania

In [97]:
display_data(sim_grants_30_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita($)  Population  
Jurisdiction ACLG Year                                                    
Tasmania     RAL  2025                                   0.0     48472.0  
                  2026                                   0.0     48641.0  
                  2027                                   0.0     48810.0  
                  2028                                   0.0     48978.0  
             RAM  2025                                   0.0     27678.0  
                  2026                                   0.0     27773.0  
                  2027                                   0.0     27869.0  
                  2028                                   0.0     27966.0  
             RAS  2025                                   0.0      3122.0  
                  2026                                   0.0      3133.0  
                  2027                                   0.0      3144.0  
                  2028                                   0.0      3154.0  
             RAV  2025                                   0.0    113763.0  
                  2026                                   0.0    114158.0  
                  2027                                   0.0    114553.0  
                  2028                                   0.0    114948.0  
             UCC  2025                                   0.0     53264.0  
                  2026                                   0.0     53449.0  
                  2027                                   0.0     53634.0  
                  2028                                   0.0     53819.0  
             UFM  2025                                   0.0    149166.0  
                  2026                                   0.0    149683.0  
                  2027                                   0.0    150201.0  
                  2028                                   0.0    150718.0  
             UFS  202

### Northern Territory

In [98]:
display_data(sim_grants_30_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTM  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTX  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   UCC  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   UFS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   URM  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   URS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   

                              Expected Change in GPG Per Capita($)  Population  
Jurisdiction       ACLG Year                                                    
Northern Territory RTL  2025                                   0.0     52234.0  
                        2026                                   0.0     52640.0  
                        2027                                   0.0     52843.0  
                        2028                                   0.0     53250.0  
                   RTM  2025                                   0.0      4380.0  
                        2026                                   0.0      4415.0  
                        2027                                   0.0      4432.0  
                        2028                                   0.0      4466.0  
                   RTS  2025                                   0.0       488.0  
                        2026                                   0.0       492.0  
                        2027                                   0.0       494.0  
                        2028                                   0.0       498.0  
                   RTX  2025                                   0.0       175.0  
                        2026                                   0.0       176.0  
                        2027                                   0.0       177.0  
                        2028                                   0.0       178.0  
                   UCC  2025                                   0.0     89822.0  
                        2026                                   0.0     90521.0  
                        2027                                   0.0     90871.0  
                        2028                                   0.0     91570.0  
                   UFS  2025                                   0.0     24609.0  
                        2026                                   0.0     24800.0  
                        2027                 